<a href="https://colab.research.google.com/github/pikanaeri/plm-model-comparison/blob/main/extracting-embeddings/extraction_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template Code for Extracting Averaged and Final Embeddings for each Model
- Stores the embeddings as .pkl files
- Final embeddings are stored in lists of vectors
- Will need the PHROGs annotation table: https://storage.cloud.google.com/plm-model-comparison/PHROG_index.tsv

## Setting Up Virtual Machine

In [ ]:
#@title Tmux Setup (optional; so that your background processes save)
sudo apt-get update -qq
sudo apt-get install -y tmux
tmux

In [ ]:
#@title Downloading Anaconda
#@markdown * If a popup menu appears, chose the "package maintainer's version" option
#@markdown * A menu containing all of Anaconda's terms of service will appear, go through these and select "yes" to download Anaconda to your computer
#@markdown * An older version of Anaconda (2022) will be downloaded, keep this in mind when checking for compatible packages
sudo apt-get update
sudo apt-get install bzip2 libxml2-dev
sudo apt upgrade

sudo apt-get install wget
wget https://repo.anaconda.com/archive/Anaconda3-2022.05-Linux-x86_64.sh
bash Anaconda3-2022.05-Linux-x86_64.sh
rm Anaconda3-2022.05-Linux-x86_64.sh

In [ ]:
#@title Allocating Memory to Linux
#@markdown Code from: https://linuxize.com/post/create-a-linux-swap-file/
sudo fallocate -l 15G /swapfile
sudo chmod 600 /swapfile
ls -lh /swapfile
sudo mkswap /swapfile
sudo swapon /swapfile


#@markdown Append "/swapfile swap swap defaults 0 0" to the /etc/fstab file using a text editor of your choice
sudo nano /etc/fstab
/swapfile swap swap defaults 0 0

In [ ]:
#@title Installing Github
sudo apt-get install git
sudo apt install python3-pip

python3 --version
pip3 --version

In [ ]:
#@title Cloning PHROGs Database; files can also be found under the Fasta Files category at https://phrogs.lmge.uca.fr/
git clone https://github.com/pikanaeri/Extracting-3Di-Embeddings-from-Protein-Sequences.git

In [ ]:
#@title Installing Dependencies
pip install transformers
pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116

mkdir final_embeddings
mkdir final_average_embeddings
python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 17.9 MB/s eta 0:00:00


In [ ]:
#@title Import any Dependencies and Packages
import torch
import re
import time
import gc
import numpy as np
import pickle as pkl

In [ ]:
#@title Set up Model and change it to GPU if available; the process for doing this will vary by model
#@markdown Embeddings were stored as Numpy arrays in lists

model = #Initialize model here
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer = #Initialize tokenizer and/or encoders here




def embed_family(Prev_dir, File_Name, max_prot=-1, max_gsz=5, max_sz=-1):
  reader = open(Prev_dir + File_Name + ".faa", "r")
  sequence_examples = []
  final_embedding = []
  num_proteins = 0
  gsz = 0
  while True:
    name = reader.readline().strip()
    if name == '':
      break
    sequence = reader.readline().strip()
    sequence = sequence.replace('-', 'X')
    if max_sz == -1 or len(sequence) <= max_sz:
      sequence_examples.append(sequence)
      num_proteins += 1
      gsz += 1
      if max_prot != -1 and num_proteins >= max_prot:
        break
    if gsz >= max_gsz:
      #embed sequence_examples here according to model instructions
      gc.collect()
      #append the final embeddings to the final_embedding array
      sequence_examples = []

  if len(sequence_examples) > 0:
    if gsz >= max_gsz:
      #embed sequence_examples here according to model instructions
      gc.collect()
      #append the final embeddings to the final_embedding array
      sequence_examples = []
      num_proteins += 1

  reader.close()
  outp_dir = File_Name + ".pkl"
  final_embedding = np.array(final_embedding)
  with open(outp_dir, 'wb') as f:
    pkl.dump(final_embedding, f)
  print(final_embedding)
  print(len(final_embedding))
  print(len(final_embedding[0]))
  outp_dir = File_Name + "_averaged.pkl"
  av_embedding = np.mean(final_embedding, axis=0)
  print(av_embedding)
  with open(outp_dir, 'wb') as f:
    pkl.dump(av_embedding, f)




def nav_dir(rg_st=-1, family_cap=-1, prot_cap=-1, sz_cap=-1, File_Name = "PHROG_index.tsv"):
  reader = open(File_Name, "r")
  labels = reader.readline().strip().split("\t")
  cnt = rg_st
  while True:
    line = reader.readline()
    if line == '':
      break
    information = line.strip().split("\t")
    phrog_num = int(information[0].split("phrog_")[1])
    if phrog_num >= rg_st:
      tm_start = time.perf_counter()
      phrog_file = "phrog_" + str(phrog_num)
      embed_family("", phrog_file, max_prot=prot_cap, max_sz=sz_cap)
      cnt += 1
      if family_cap != -1 and cnt >= family_cap:
        break
      print(cnt, " extracted!")
      print("took ", str((time.perf_counter()-tm_start)/60), " minutes")
  reader.close()


nav_dir() #range_start indicates the starting PHROG family, family_cap determines the ending PHROG family, prot_cap indicates the number of proteins extracted per family, sz_cap indicates the maximum sequence length